In [5]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [8]:
using DifferentialEquations
using BenchmarkTools
using CSV, DataFrames
using Tables
using StaticArrays
using DelimitedFiles

In [10]:
function write_row_vector!(A, i, row)
    for j in 1:3
        A[i,j] = row[j]
    end
end

write_row_vector! (generic function with 1 method)

In [45]:
function quad_2d!(d_state, state, params, t)

    # extract the state
    X = @view state[1:6]

    # extract the control input
    U = @view state[7:end]
    
    d_state .= 0.0
end

tspan = (0.0, 60.0)
const dt::Float64 = 0.01

0.01

# Method 2: Periodic Callback - Preallocate logging matrix

In [46]:
control_cb = PeriodicCallback(dt, initial_affect=true, save_positions=(false, true)) do integrator
    n_timestep = convert(Int, round(integrator.t/dt)+1)

    X = @view integrator.u[1:6]
    
    write_row_vector!(integrator.p.log_matrix, n_timestep, integrator.u)
end;

In [48]:
let

n_rows::Int = length(tspan[1]:dt:tspan[2])
n_cols = 3
log_matrix =zeros(n_rows,n_cols)
    
params = (;log_matrix=log_matrix)


u0 = ones(8)
prob = ODEProblem(quad_2d!, u0, tspan, params, callback=control_cb);
sol = solve(prob, Tsit5(),save_everystep=false, dense=false);

@time sol = solve(prob, Tsit5(),save_everystep=false, dense=false);

@time CSV.write("log_no_alloc.csv",  Tables.table(log_matrix), writeheader=false)

end;

  0.012238 seconds (6.06 k allocations: 999.234 KiB)
  0.001448 seconds (55 allocations: 4.003 MiB)


# Method 3: No Callback with fixed timestep (use default logging from DifferentialEquations.jl)

In [186]:
let
    
prob = ODEProblem(lorenz!, u0, tspan)
@time sol = solve(prob, Tsit5(), dt=0.01, adaptive=false, dense=false);

end;

  0.007566 seconds (6.05 k allocations: 846.906 KiB)


In [206]:
let

u0 = ones(8)
tspan = (0.0, 100.0)
prob = ODEProblem(lorenz!, u0, tspan)
@time sol = solve(prob, Tsit5(), abstol=1e-8, reltol=1e-8, save_everystep=false, dense=false);

end;

  0.006877 seconds (49 allocations: 4.891 KiB)
